In [1]:
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import os
import pickle
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler,scale,MinMaxScaler
from sklearn.metrics import mean_squared_error

from batt_descriptor.batt_describe import *
from model.knn import *
from predictor.batt_predict import *

ModuleNotFoundError: No module named 'batt_descriptor'

# Testing descriptor functions #

In [ ]:
# Testing descriptor functions - 1
df_encoded_ACE = ohe_ACE(df_battery)
df_encoded_ACE

In [ ]:
# Testing descriptor functions - 2
df_enbattery = ohe_dataframe(df_battery)
df_enbattery

# Testing model functions #

In [ ]:
# Testing model function - 1
train,test,X_train,y_train,X_test,y_test = data_split(df_battery, 0.2, 'Discharge_Capacity (Ah)', 66)

In [ ]:
# Testing model function - 2
kf = kfold(10,66)

In [ ]:
# Splitting data 
train,test,X_train,y_train,X_test,y_test = data_split(df_battery, 0.2, 'Discharge_Capacity (Ah)', 66)
X_train_scaled, X_test_scaled = data_scale(X_train, X_test)
X_test_scaled

In [ ]:
# Hyperparameter optimization
alg, n_neigh, weight = grid_knn_hp(1, 51, df_battery, 'Discharge_Capacity (Ah)')
print(alg, n_neigh, weight)

In [ ]:
# training RMSE vibe check
knn_train_RMSE_avg = knn_train(df_battery, 'Discharge_Capacity (Ah)')
knn_train_RMSE_avg

In [ ]:
# testing RMSE vibe check
knn_test_RMSE = knn_test(df_battery, 'Discharge_Capacity (Ah)')
knn_test_RMSE

# And finally, predictor #

In [ ]:
X_set = X_set_in(df_battery)
X_set

In [ ]:
X_set = X_set_en(df_battery)
X_set

In [ ]:
X_bat, y_bat = df_prep(df_battery, 'Discharge_Capacity (Ah)')
print(X_bat)
print(y_bat)

In [ ]:
y_pred = battery_predictor(df_battery, 'Discharge_Capacity (Ah)')
y_pred

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
report = report_gen(df_battery)
report

In [ ]:
def backend_to_frontend(df_battery, user_in, report):
    """ 
    This function pulls the data from the report
    based on the user input from the front end.
    User input should be a list.
    """
    report = report_gen(df_battery)
    data_indices = []
    n, m = report.shape
    
    for i in range(n):
        if report['cathode'][i]==user_in[1] and \
        report['temperature'][i]==user_in[3] and \
        report['discharge_crate'][i]==user_in[4]:
            data_indices.append(i)
    
    pulled_data = pd.DataFrame(columns = report.columns)
    
    for j in range(len(data_indices)):
        pulled_data.loc[j] = report.iloc[data_indices[j], :]
        
    return pulled_data

In [ ]:
# this function will take the report and the user input list as inputs
# output will be pulled data
user_in = ['graphite', 'LFP', 50, 25, 1.0]
data_indices = []
n, m = report.shape

# here's a code to extract all the values 
# currently it only extractgs the values for one cycle number
# but we can extend it to more later by doing something like:
# append the next two values as well
# if there are no more points, just show the last three points 

for i in range(n):
    if report['cathode'][i]==user_in[1] and \
    report['temperature'][i]==user_in[3] and \
    report['discharge_crate'][i]==user_in[4]:
        data_indices.append(i)
        
print(data_indices)

pulled_data = pd.DataFrame(columns = report.columns)

# we can edit this part to extract the other values as well
# we also need to tie in the electrolyte to use as the label
# we thinking ahead with the plotting, we can avoid fixing the connected plot lines
# if we arrange this data structure smart enough

for j in range(len(data_indices)):
    pulled_data.loc[j] = report.iloc[data_indices[j], :]
    
pulled_data

# Below is visualization testing #

In [ ]:
N = 70

fig = go.Figure(data=[go.Mesh3d(x=(70*np.random.randn(N)),
                   y=(55*np.random.randn(N)),
                   z=(40*np.random.randn(N)),
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  )])

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[-100,100],),
                     yaxis = dict(nticks=4, range=[-50,100],),
                     zaxis = dict(nticks=4, range=[-100,100],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))

fig.show()

In [ ]:
x = [1,1,1] # input, strings (ex: 'NMC')
y = [50,100,150] # input
z = [13, 69, 21] # output

fig = go.Figure(data=go.Scatter3d(
    x=x, y=y, z=z,
    marker=dict(
        size=10,
        color = x
    ),
    line=dict(
        color=x,
        width=3,
        dash='dash'
    ),
    name='battery 1', text = 'battery 1'
))

fig.update_layout(title ={'text' :'help me', 
                          'x' : 0.5},
                 scene = dict(
                     xaxis = dict(
                         nticks = 3,
                         ticktext = ['NMC', 'NFP', 'LCA'],
                         tickvals = [0, 1, 2],
                     title = 'help'))
                 )
fig.show()

In [ ]:
pulled_data.columns

In [ ]:
# first 10 points are A123
elec_a123 = pulled_data.iloc[0:10, :]

# next 10 are Pan
elec_pan = pulled_data.iloc[10:20, :]

# last ten are LGC
elec_LGC = pulled_data.iloc[20:30, :]

fig = go.Figure(data=go.Scatter3d(
    x=elec_a123['electrolyte'], 
    y=elec_a123['Cycle'], 
    z=elec_a123['Charge_Capacity (Ah)'],
    marker=dict(
        size=10,
        color = '#5271FF'
    ),
    line=dict(
        color='#5271FF',
        width=3,
        dash='dash'
    ),
    name='A123'
))


fig.add_trace(
    go.Scatter3d(
        x=elec_pan['electrolyte'],
        y=elec_pan['Cycle'],
        z=elec_pan['Charge_Capacity (Ah)'],
        marker=dict(
            size=10,
            color = '#FF5757'
        ),
        line=dict(
            color='#FF5757',
            width=3,
            dash='dash'
        ),
        name='Panasonic'
    )
)


fig.add_trace(
    go.Scatter3d(
        x=elec_LGC['electrolyte'], 
        y=elec_LGC['Cycle'],
        z=elec_LGC['Charge_Capacity (Ah)'],
        marker=dict(
            size=10,
            color = '#FFBD59'
            #color = '#545454'
        ),
        line=dict(
            color = '#FFBD59',
            #color='#545454',
            width=3,
            dash='dash'
        ),
        name='LG Chem'
    )
)


fig.update_layout(title ={'text' :user_in[1] + ' Cathode Charge Capacity', 
                          'x' : 0.5},
                  autosize=False,
                  width=800,
                  height=600,
                  font = dict(size = 15),
                  scene = dict(
                      xaxis = dict(
                          nticks = 3,
                          title = 'Electrolyte'),
                      yaxis = dict(
                          title = 'Cycles'),
                      zaxis = dict(
                          title = 'Charge Capacity [Ah]')
                      )
                  )

fig.update_yaxes(automargin=True)

fig.show()

In [ ]:
fig1 = go.Figure(data=go.Scatter(x=elec_a123['Cycle'], 
                                 y=elec_a123['Coulombic_Efficiency (%)'],
                                 marker=dict(
                                 size=10,
                                 color = '#5271FF'
                                ),
                                line=dict(
                                    color='#5271FF',
                                    width=3,
                                    dash='dash'
                                ),
                                name='A123')
                )

fig1.add_trace(go.Scatter(x=elec_pan['Cycle'], 
                                 y=elec_pan['Coulombic_Efficiency (%)'],
                                 marker=dict(
                                     size=15,
                                     color = '#FF5757',
                                     symbol = 'triangle-up'
                                 ),
                                 line=dict(
                                     color='#FF5757',
                                     width=3,
                                     dash='dash'
                                 ),
                                 name='Panasonic'
                         )
              )

fig1.add_trace(go.Scatter(x=elec_LGC['Cycle'], 
                          y=elec_LGC['Coulombic_Efficiency (%)'],
                          marker=dict(
                              size=10,
                              color = '#FFBD59',
                              #color = '#545454',
                              symbol = 'square'
                          ),
                          line=dict(
                              color = '#FFBD59',
                              #color='#545454',
                              width=3,
                              dash='dash'
                          ),
                          name='LG Chem'
                         )
              )

fig1.update_layout(title ={'text' :user_in[1] + ' Coulombic Efficiency', 
                          'x' : 0.45},
                   font = dict(size = 15),
                   xaxis = dict(
                       title = 'Number of Cycles'),
                   yaxis = dict(
                       title = 'CE (%)')
                  )
fig1.show()

In [ ]:
x = [0,0,0,1,1,1,2,2,2] # input, strings (ex: 'NMC')
y = [50,100,150, 50,100,150, 50, 100, 150] # input
z = [13, 69, 21, 5, 25,50, 60, 40, 20] # output

df = pd.DataFrame([x,y,z]).T

# to avoid the connected lines between data
# sort data
# if = 1, lca, etc

fig = go.Figure(data=go.Scatter3d(
    x=df[0], y=df[1], z=df[2],
    marker=dict(
        size=10,
        color = x
    ),
    line=dict(
        color=x,
        width=3,
        dash='dash'
    ),
    name='battery 1', text = 'battery 1'
))

fig.update_layout(title ={'text' :'help me', 
                          'x' : 0.5},
                 scene = dict(
                     xaxis = dict(
                         nticks = 3,
                         ticktext = ['NMC', 'NFP', 'LCA'],
                         tickvals = [0, 1, 2],
                     title = 'help'))
                 )
fig.show()

df.head()

In [ ]:
x = [1,1,1] # input, strings (ex: 'NMC')
y = [50,100,150] # input
z = [13, 69, 21] # output

fig = px.scatter_3d(x=x, y=y, z=z,
                    color=x, 
                    labels = {'x': 'Battery System', 'y': 'Cycle Number',
                              'z': 'Output'})

fig.update_layout(title ={'text' :'help me', 
                          'x' : 0.5},
                 scene = dict(
                     xaxis = dict(
                         nticks = 3,
                         ticktext = ['NMC', 'NFP', 'LCA'],
                         tickvals = [0, 1, 2]))
                 )

fig.show()

In [ ]:
x = [1,1,1] # input, strings (ex: 'NMC')
y = [50,100,150] # input
z = [13, 69, 21] # output

fig = go.Figure(data=px.line_3d(
    x=x, y=y, z=z,
    title='spinny boi',                    
    labels = {'x': 'Battery System', 'y': 'Cycle Number', 'z': 'Output'},
    markers = True
     ))

fig.update_layout(title = {'text' :'help me', 'x' : 0.5},
                 scene = dict(
                     xaxis = dict(
                         nticks = 3,
                         ticktext = ['NMC', 'NFP', 'LCA'],
                         tickvals = [0, 1, 2]))
                 )

fig.show()